Group9: Kelly Berdelle, Jason Houghton, Yuebo Li, Qinya Wang, Gaoshuang Zhu

In [1]:
import tweepy
import csv
from tweepy import Stream
from tweepy.streaming import StreamListener
import re
from textblob import TextBlob

In [2]:
with open('auth.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
authdata = {}   
for element in your_list:
    authdata[element[0]] = element[1]
authdata

{'Access_Token': '349710349-SCQO1KIi3VSQytQinCHvCzKgWlS9zMdEsOTCvqA7',
 'Access_Token_Secret': 'LOZKO4mojBFu2nCUn7kxoTXJQE3gnI7ZXaDH4hJuVeH5c',
 'Consumer_Key': '1vnncA8ZE0lJC4U0cqKJOeX3n',
 'Consumer_Secret': 'cKawPhcPzGrSZneIFNUnHOpAmLXNTxN7lVb230ajyoTi1hZ2cK'}

In [14]:
Consumer_Key = authdata['Consumer_Key']
Consumer_Secret = authdata['Consumer_Secret']
Access_Token = authdata['Access_Token']
Access_Token_Secret = authdata['Access_Token_Secret']
auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret)
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth)

Set up the Listener

In [95]:
def gettweets(search, loc, filename):
    class MyListener(StreamListener):
        def __init__(self, api=None):#this fuction limits the number of tweets we get
            super(MyListener, self).__init__()
            self.num_tweets = 0
        def on_status(self, status):#this function overrides the on_status method and writes tweets to JSON file       
            try:
                text = status.text.encode("utf8","ignore")
                text = str(text)+'\n'
                self.num_tweets += 1
                if self.num_tweets < 500:
                    with open(filename, 'a') as f:
                        
                            f.write(text[2:])
                            return True
                else:
                    return False
            except (TypeError,AttributeError) as e:
                print(str(e))
            return True
 
        def on_error(self, status):#this fuction override the on_error method so that we can handle errors from the properly
            if status_code == 420:
                return False
    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(track=[search], locations=loc)

Define funtions that cleans the text

In [96]:
def remove_punctuation(s):    
    punctuation = "\\#$%&'()*+-/:;<=>@[\\]^_`{|}~"
    s_sans_punct = ""
    for letter in s:
        if (letter not in punctuation) and (letter in "abcdefghijklmnopqrstuvwxyz "):
            s_sans_punct += letter
    return s_sans_punct

def getLowerCaseText(status_texts):
    lowered_texts = ''
    for texts in status_texts:
        try: 
            mytext = str(texts.lower())
            lowered_texts = lowered_texts + mytext
        except:
            pass
    return lowered_texts

def removeUrl(text):
    s = re.sub(r"http\S+", "", text)
    return s

In [17]:
def clean(file):   
    with open(file, 'r') as f:
        text = []
        for line in f:
            s = removeUrl(line)
            lowered_texts = getLowerCaseText(s)
            cleantext = remove_punctuation(lowered_texts)
            text.append(cleantext)
    return text

Define the function that generates scores

In [97]:
def analysis(text):
    count = 0
    pol = 0
    for line in text:
        blob = TextBlob(line)
        count+=1
        pol += blob.sentiment.polarity
    return 50*(1 + pol/count)

**If error happens, wait for about half minute and run again. It will finaly worked!**

In [55]:
gettweets('trump',[-86.8,41.87,-82.54,45.35],'tr_mi.json')
text = clean('tr_mi.json')
t_mi_score = analysis(text)
print (t_mi_score)

53.27999077761102


In [56]:
gettweets('clinton',[-86.8,41.87,-82.54,45.35],'cl_mi.json')
text = clean('cl_mi.json')
c_mi_score = analysis(text)
print (c_mi_score)

52.12172339508261


In [57]:
gettweets('trump',[-115.7,35.6,-114,38, -118.5,38,-114,42],'tr_nv.json')
text = clean('tr_nv.json')
t_nv_score = analysis(text)
print (t_nv_score)

54.70039224673493


In [58]:
gettweets('clinton',[-115.7,35.6,-114,38, -118.5,38,-114,42],'cl_nv.json')
text = clean('cl_nv.json')
c_nv_score = analysis(text)
print (c_nv_score)

53.13686430701712


In [99]:
gettweets('trump',[-90.5,42.5,-87,45, -92.5,45,-88,45.7],'tr_wi.json')
text = clean('tr_wi.json')
t_wi_score = analysis(text)
print (t_wi_score)

51.01271728847044


In [98]:
gettweets('clinton',[-90.5,42.5,-87,45, -92.5,45,-88,45.7],'cl_wi.json')
text = clean('cl_wi.json')
c_wi_score = analysis(text)
print (c_wi_score)

50.19404814923628
